In [1]:
import pandas as pd

In [2]:
def remover_espacos_colunas(df):
    """
    Remove espaços em branco dos nomes das colunas de um DataFrame.
    
    Argumentos:
    df (DataFrame): DataFrame do pandas.
    
    Retorna:
    DataFrame: DataFrame com os espaços em branco removidos dos nomes das colunas.
    """
    for coluna in df.columns:
        if isinstance(coluna, str):  
            novo_nome = coluna.strip().replace(' ', '')  
            df.rename(columns={coluna: novo_nome}, inplace=True)  
    return df
def lista_colunas_unicas(dicionario_dataframes):
    """
    Retorna uma lista de colunas únicas presentes nos DataFrames do dicionário, excluindo 'ESTUDANTE'.
    
    Argumentos:
    dicionario_dataframes (dict): Dicionário contendo DataFrames do pandas.

    Retorna:
    list: Lista de colunas únicas.
    """
    colunas_unicas = set()
    for df_nome, df in dicionario_dataframes.items():
        colunas_unicas.update([col.replace('.1', '').strip() for col in df.columns])
    colunas_unicas.discard("ESTUDANTE")
    return list(colunas_unicas)

def extrair_elementos_para_dict(lista):
    """
    Extrai os elementos de uma lista e os insere em um dicionário com índices numerados a partir de 1.
    
    Argumentos:
    lista (list): Lista de elementos.

    Retorna:
    dict: Dicionário contendo os elementos numerados.
    """
    dict_resultante = {}
    for idx, dic in enumerate(lista, start=1):
        dict_resultante[idx] = dic
    return dict_resultante

def extrair_abas_excel(nome_arquivo):
    """
    Extrai abas de um arquivo Excel para um dicionário de DataFrames, removendo espaços em branco dos nomes das colunas.
    
    Argumentos:
    nome_arquivo (str): Nome do arquivo Excel.

    Retorna:
    dict: Dicionário contendo abas do Excel como DataFrames do pandas.
    """
    xls = pd.ExcelFile(nome_arquivo)
    abas_dict = {}
    for aba_nome in xls.sheet_names:
        df = xls.parse(aba_nome)
        aba_nome = aba_nome.strip()
        abas_dict[aba_nome] = remover_espacos_colunas(df)
    return abas_dict

def gerar_alunos(lista_de_listas):
    """
    Gera uma lista de alunos únicos a partir de uma lista de listas de nomes de alunos.
    
    Argumentos:
    lista_de_listas (list): Lista de listas contendo nomes de alunos.

    Retorna:
    list: Lista de alunos únicos.
    """
    alunos_unicos = set()
    for lista in lista_de_listas:
        alunos_unicos.update(lista)
    return list(alunos_unicos)


In [3]:
def calcular_participacao_aluno(df, flag_atraso):
    """
    Calcula a participação de um aluno em aulas com um determinado tipo de atraso em um DataFrame.

    Argumentos:
    df (DataFrame): DataFrame contendo os registros de participação do aluno.
    flag_atraso (str): Flag que indica o tipo de atraso a ser considerado.

    Retorna:
    Series: Série contendo a participação do aluno em aulas com o tipo de atraso especificado.
    """
    df_alvo = pd.DataFrame(df.iloc[:, 1:16] == flag_atraso)
    return df_alvo.replace({True: 1, False: 0}).sum(axis=1)

def calcular_participacao_materia(df, flag_atraso):
    """
    Calcula a participação dos alunos em aulas com um determinado tipo de atraso em um DataFrame de matéria.

    Argumentos:
    df (DataFrame): DataFrame contendo os registros de participação dos alunos na matéria.
    flag_atraso (str): Flag que indica o tipo de atraso a ser considerado.

    Retorna:
    dict: Dicionário contendo o número de participações dos alunos em aulas com o tipo de atraso especificado para cada matéria.
    """
    df_alvo = pd.DataFrame(df.iloc[:, 1:] == flag_atraso).sort_index(axis=1)
    return df_alvo.replace({True: 1, False: 0}).sum(axis=0).to_dict()


In [4]:
def gerar_presenca_alunos(turma,alunos_turma):
    """
    Gera um DataFrame contendo a presença dos alunos em cada semana de uma determinada turma.

    Argumentos:
    turma (str): Identificador da turma.
    alunos_turma (list): Lista de alunos da turma.

    Retorna:
    DataFrame: DataFrame contendo a presença dos alunos em cada semana da turma.
    """
    base_alunos_sala_parcial = {}
    id = 1 
    for aluno in alunos_turma:
        for semana in range(1, 4):
            try:
                df_semana = turmas_frequencia_marco_2023[f'Sala {turma} - Semana {semana}'].query(f"ESTUDANTE == '{aluno}'")
                base_alunos_sala_parcial[id] = {
                    'ESTUDANTE': aluno,
                    'TURMA': turma,
                    'SEMANA': semana,
                    'PRESENCA': int(calcular_participacao_aluno(df_semana, 1)), 
                    'FALTAS': int(calcular_participacao_aluno(df_semana, 0)),
                    'ATRASOS15': int(calcular_participacao_aluno(df_semana, '0*')),
                    'ATRASOS30': int(calcular_participacao_aluno(df_semana, '0**'))
                }
            except:
                base_alunos_sala_parcial[id] = {
                    'ESTUDANTE': aluno,
                    'TURMA': turma,
                    'SEMANA': semana,
                    'PRESENCA': -1,
                    'FALTAS': -1,
                    'ATRASOS15': -1,
                    'ATRASOS30': -1
                }
            id += 1
    return pd.DataFrame(base_alunos_sala_parcial).T

# Extração e processamento da frequencia das turmas 

### Carregando todas as frequências e extraindo todas as disciplinas lecionadas no mês

In [5]:
path_frequencia_marco_2023 = 'data/raw/Frequencia_Marco_2023.xlsx'  
turmas_frequencia_marco_2023 = extrair_abas_excel(path_frequencia_marco_2023)
 

In [6]:
DISCIPLINAS = lista_colunas_unicas(turmas_frequencia_marco_2023)

## Gerando a base de disciplinas

### Criando um dicionário de mapeamento para as semanas e turmas

In [7]:
salas = ['A','B']
semanas = [1,2,3]
arquivos = list(turmas_frequencia_marco_2023.keys())
resultado_disciplinas = {}
cod = 1

In [8]:
dados_disciplinas = {}
for semana in semanas:
    for sala in salas:
        chave = f"{semana:02d}_{sala}"
        arquivo = f'Sala {sala} - Semana {semana}'
        dados_disciplinas[chave] = {'sala': sala, 'semana': semana, 'arquivo': arquivo}

### Extração de todas as disciplinas

Após o levantamento de todas as disciplinas no mês e feito a combinação dos datasets é feito a extração dos dados por disciplinas. Como nem todas as semanas possuem a mesma quantidade de aulas na semana nem que todas as matérias são lecionadas na semana é feito a adaptação para esse caso.

In [9]:
resultados = []

In [10]:
for id in list(dados_disciplinas.keys()):
    arquivo_base =  dados_disciplinas[id]['arquivo']
    for disciplina in DISCIPLINAS:
        discp_semana = {}  
        disciplina_extra = disciplina + '.1'
        disciplinas_semana = list(turmas_frequencia_marco_2023[arquivo_base].columns)
        if disciplina in disciplinas_semana:
            qtd_aulas = 1
            atraso15 = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0*')[disciplina]
            atraso30 = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0**')[disciplina]
            faltas = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 0)[disciplina]
            presencas = calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 1)[disciplina]
            if disciplina_extra in disciplinas_semana:
                qtd_aulas = qtd_aulas + 1
                atraso15 = (atraso15 + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0*')[disciplina_extra])
                atraso30 = (atraso30 + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], '0**')[disciplina_extra])
                faltas = (faltas + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 0)[disciplina_extra])
                presencas = (presencas + calcular_participacao_materia(turmas_frequencia_marco_2023[arquivo_base], 1)[disciplina_extra])         
        else:
            qtd_aulas = 0
            atraso15 = 0
            atraso30 = 0
            faltas = 0
            presencas = 0
        discp_semana[cod] = {
            'ID': id,
            'SEMANA': dados_disciplinas[id]['semana'],
            'SALA': dados_disciplinas[id]['sala'],
            'DISCIPLINA': disciplina,
            'QTD_AULAS_SEMANA': qtd_aulas,
            'PRESENÇAS': presencas,
            'FALTAS': faltas,
            'ATRASO15': atraso15,
            'ATRASO30': atraso30
        }
        resultados.append(discp_semana)
        cod += 1

Por fim é criado uma base com esses resultados e exportado o csv para a parte do BI.

In [11]:
tam_base = len(resultados)
base_final = {}
for num in range(1,tam_base+1):
    base_final[num] = extrair_elementos_para_dict(resultados)[num][num]
base_disciplinas = pd.DataFrame(base_final).T    
    

In [12]:
# base_disciplinas.to_csv('situacao_disciplinas.csv',index=False)

## Gerando base de alunos

Aqui é gerado uma lista de alunos únicos de todo o mês realizando a diferenciação apenas por turma. Após esse processo é chamada uma função que irá gerar a situação de presença,falta e atrasos de cada estudante. Se aquele aluno não for encontrado a flag -1 será adicionada.

In [13]:
alunos_sala_a = []
alunos_sala_b = []   
for semana in range(1, 4):
    alunos_sala_a.append(list(turmas_frequencia_marco_2023[f'Sala A - Semana {semana}']['ESTUDANTE']))
    alunos_sala_b.append(list(turmas_frequencia_marco_2023[f'Sala B - Semana {semana}']['ESTUDANTE']))
    
    

In [14]:
alunos_sala_a = gerar_alunos(alunos_sala_a)
alunos_sala_b = gerar_alunos(alunos_sala_b)

Exportando arquivos CSV

In [15]:
# gerar_presenca_alunos('A',alunos_sala_a).to_csv('alunos_sala_a.csv',index=False)

In [16]:
# gerar_presenca_alunos('B',alunos_sala_b).to_csv('alunos_sala_b.csv',index=False)